В данном материале разберёмся с тем, как строить рекомендательные системы на основе сингулярного разложения матриц. 

Начнём с постановки задачи. Что такое вообще рекомендательная система? Это алгоритм, который предлагает нашим пользователям/клиентам товары/контент с целью увеличения счастья пользователя или выручки.
Подходы к построению таких систем могут быть разными:

- Рекомендовать самые популярные товары/фильмы/новости/музыку;
- Колоборативная фильтрация на основе user-based/item-based (метрические методы);
- Рекомендации на основе факторизации матриц(SVD-разложение);
- Можно обучить регрессор, предсказывающий выручку или рейтинг фильма для пары "Пользователь-Объект";

Я же подробно остановлюсь на третьем пункте, т.к. этот метод построения рекомендательных систем достаточно неплох: быстро работает и показывает неплохое качество рекомендаций, выявляя скрытые интересы пользователей и качества объектов (фильм, товар и т.п.).

Представим, что у нас есть небольшой онлайн-кинотеатр и мы хотим, чтобы пользователи были довольны и смотрели фильмы у нас, а не на других ресурсах. Создадим небольшую матрицу рейтингов наших фильмов, на основании оценок пользователей.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

In [2]:
#Названия фильмов
f = ['Терминатор', 'Робокоп', 'Рэмбо', 'Джеймс Бонд','Властелин Колец', 'Хоббит', 'Гарри Поттер', 
     'Американский Пирог', 'Мальчишник в Вегасе', 'Мстители', 'Супермен'] 

In [5]:
#Наши пользователи
u = ['Вася','Петя','Саша','Женя','Маша','Оля','Лена','Ваня','Ира']

In [3]:
#Рейтинг фильмов от пользователей. 0 - значит фильм не был просмотрен.
fu = np.array([[9,0,0,6,4,3,5,7,8,6,5],
              [0,0,0,8,10,0,8,5,6,2,0],
              [8,0,5,7,4,0,5,7,8,8,5],
              [5,2,0,4,10,0,9,5,0,0,0],
              [0,0,2,5,7,0,0,9,0,4,0],
              [2,3,0,0,0,7,9,0,5,0,3],
              [5,0,3,8,8,0,0,7,0,2,4],
              [8,7,0,6,5,4,0,8,0,10,0],
              [0,0,2,5,10,0,0,0,9,0,3]])

In [6]:
#Сделаем датафрейм из f,u и fu.
R = pd.DataFrame(fu,u,f)

In [7]:
R

,Терминатор,Робокоп,Рэмбо,Джеймс Бонд,Властелин Колец,Хоббит,Гарри Поттер,Американский Пирог,Мальчишник в Вегасе,Мстители,Супермен
Вася,9,0,0,6,4,3,5,7,8,6,5
Петя,0,0,0,8,10,0,8,5,6,2,0
Саша,8,0,5,7,4,0,5,7,8,8,5
Женя,5,2,0,4,10,0,9,5,0,0,0
Маша,0,0,2,5,7,0,0,9,0,4,0
Оля,2,3,0,0,0,7,9,0,5,0,3
Лена,5,0,3,8,8,0,0,7,0,2,4
Ваня,8,7,0,6,5,4,0,8,0,10,0
Ира,0,0,2,5,10,0,0,0,9,0,3


Вот теперь и встаёт вопрос, какой предложить фильм, чтобы наш пользователь хорошо провёл вечер? Немного поискав на просторах интернета, находим статью о том, что один большой онлнайн-кинотеатр **Netflix** в 2006 году объявил соревнование по разработке рекомендательной системы фильмов. За решение, которое улучшит качество рекомендаций на 10%, была объявлена награда в 1 млн долларов. В итоге в 2011 году такое решение было получено, а работало оно на основе **SVD** - Singular Value Decomposition или Сингулярном Разложении Матриц, если по-русски.

### Немного линейной алгебры.
Смысл SVD состоит в том, что прямоугольную матрицу A мы можем разложить на 3 матрицы: U, Ʃ и V.

<img src="SVD_img_2.png" style="width: 300px">

Матрицы U и V ортогональные, а Ʃ — диагональная (хотя и не квадратная). 

<img src="SVD_img_3.png" style="width: 600px">

В случае разреженной матрицы, которой обычно является рейтинг фильмов, можно использовать усечённое разложение (truncated SVD), где мы оставляем только d первых чисел $\lambda$. В итоге получаем разложение матрицы A', которое хорошо приближает исходную матрицу A.


<img src=SVD_img_5.png style="width: 300px">

Теперь применим это к нашей рекомендательной системе. У нас есть матрица оценок R, давайте сделаем её сингулярное разложение.
Первые две матрицы перемножим, получим также матрицу $n$ x $d$, в итоге получится такое разложение.
<img src="SVD_img_4.png" style="width: 600px">

Размерность **d** в данном случае отвечает за количество скрытых свойств/интересов у наших пользователей и фильмов, $\hat r \tiny ui$ - рейтинг фильма, который мы получили перемножив матрицы **U** и **V**.

Таким образом, мы научились приближать исходную матрицу оценок матрицей, полученной на основе перемножения двух матриц, отвечающих за интересы пользователей и свойства фильмов. Но проблема в том, что матрица **R** у нас разреженная, и мы бы хотели бы как раз заполнить пустые (нулевые) элементы оценками пользователей.


Как же это сделать? Для этого нам нужно приблизить уже имеющиеся оценки числами, полученными в результате перемножения матриц $U$ и $V$. Тогда мы получим и оценки для нулей нашей исходной матрицы, т.е. нужно минимизировать такую функцию:

$$\large \frac{1}{n}\Sigma_{ui} (r {\tiny ui}-\hat r {\tiny ui})^{2} \to min,$$ где суммирование идёт по ненулевым индексам $u,i$ нашей исходной матрицы оценок, а $n$ - количество оценок в матрице.

Так это же задача регрессии с квадратичной функций потерь **MSE**. **MSE** выбрана по двум причинам: историческая, Netflix предложили метрику **RMSE** и она лучше оптимизируется градиентным спуском.

Функция потерь есть, метод оптимизации выбран, что забыли? Регуляризатор! Воспользуемся $L{\tiny 2}$-регуляризацией: 
$\large \lambda(\Sigma_{u}p {\tiny u}^{2} + \Sigma_{i}q {\tiny i}^{2})$
Осталось посчитать градиенты для реализации поиска минимума **MSE**. Воспользуемся стохастическим градиентным спуском, поэтому будем считать градиенты на одном объекте $\large r {\tiny ui}$ из матрицы оценок. В итоге получаем такие правила обновления элементов наших матриц **U** и **V**.

$\large 
{p_{u,j}=p_{u,j} + \large \gamma (r_{ui}-\hat r_{ui}) q_{i,j} - \lambda p_{u,j}}$


$\large {q_{i,j}=q_{i,j} + \large \gamma (r_{ui}-\hat r_{ui}) p_{u,j} - \lambda q_{i,j}}$

Индексы $u,i$ - номера пользователя и фильма в наших матрицах **U** (номер строки) и **V** (номер столбца), $j$ - $j$-ая компонента векторов $\large p_{u}$ и $\large q_{i}$.

Ну что же, теперь воспользуемся Питоном и сделаем рекомендации для наших пользователей!

In [10]:
def SVD(R,d,step,lambda_reg,n_iters):
    #инициализуем наши матрицы для разложения
    U = np.zeros((R.shape[0],d))
    V = np.zeros((d,R.shape[1]))
    #начальные элементы матрица U и V будут средним рейтингом по ненулевым оценкам
    mu = R.sum()/(R!=0).sum()
    non_zero = (R!=0).sum()
    U = U + mu
    V = V + mu
    # Создадим списки, где будут индексы нулевых и ненулевых элементов матрицы R
    indx=[]
    zero_indx = []
    # Инициализируем MSE в начале и будем отслеживать в процессе обучения
    MSE_start = 0
    MSE=[]
    # Найдём индексы нулевых и ненулевых элементов
    for i in range(R.shape[0]):
        for j in range(R.shape[1]):
            if R[i][j]>0:
                indx.append([i,j])
                MSE_start+=((R[i,j]-np.dot(U[i,:],V[:,j])) ** 2) / non_zero
            else:
                zero_indx.append([i,j])
    # Сделаем градиентный спуск            
    for n in range(n_iters):
        choice = np.random.randint(0,len(indx))
        ij = indx[choice]
        for k in range(0,d):
            U[ij[0],k] = U[ij[0],k] + step * ((R[ij[0]][ij[1]] - 
                                               np.dot(U[ij[0],:],V[:,ij[1]])) * V[k,ij[1]] - 
                                              lambda_reg * U[ij[0],k])
            V[k,ij[1]] = V[k,ij[1]] + step * ((R[ij[0]][ij[1]] - 
                                               np.dot(U[ij[0],:],V[:,ij[1]])) * U[ij[0],k] - 
                                              lambda_reg * V[k,ij[1]])
            
        L=0
        for i in range(R.shape[0]):
            for j in range(R.shape[1]):
                if R[i,j]>0:
                    L+=((R[i,j]-np.dot(U[i,:],V[:,j])) ** 2)/non_zero
        MSE.append(L)    
    return U,V, MSE_start, MSE

In [11]:
A,B,M_1,M_end = SVD(R.values,2,0.01,0.1,3000)

In [9]:
R.values

array([[ 9,  0,  0,  6,  4,  3,  5,  7,  8,  6,  5],
       [ 0,  0,  0,  8, 10,  0,  8,  5,  6,  2,  0],
       [ 8,  0,  5,  7,  4,  0,  5,  7,  8,  8,  5],
       [ 5,  2,  0,  4, 10,  0,  9,  5,  0,  0,  0],
       [ 0,  0,  2,  5,  7,  0,  0,  9,  0,  4,  0],
       [ 2,  3,  0,  0,  0,  7,  9,  0,  5,  0,  3],
       [ 5,  0,  3,  8,  8,  0,  0,  7,  0,  2,  4],
       [ 8,  7,  0,  6,  5,  4,  0,  8,  0, 10,  0],
       [ 0,  0,  2,  5, 10,  0,  0,  0,  9,  0,  3]])

In [12]:
A

array([[ 0.7963929 ,  1.03708078],
       [-1.27468466,  2.49428178],
       [ 0.90870672,  1.0367286 ],
       [-1.03185159,  2.40176395],
       [-0.38535967,  1.77895421],
       [-0.73988821,  1.8195959 ],
       [-0.84373258,  2.30360539],
       [ 0.64326813,  1.09876665],
       [-0.75405362,  2.39529452]])

In [13]:
B

array([[5.11342154, 4.88811713, 3.35084727, 2.71047492, 0.08915113,
        4.41076405, 3.37169286, 4.1762283 , 4.52868804, 5.25752811,
        2.85932923],
       [3.83437355, 3.29491809, 2.05220063, 3.47339214, 3.96011801,
        4.02230872, 4.98989997, 4.3522548 , 4.76868526, 3.20876402,
        2.58129483]])

In [14]:
r_cap = np.zeros((R.shape[0],R.shape[1]))
for i in range(R.shape[0]):
    for j in range(R.shape[1]):
        r_cap[i,j]=np.dot(A[i,:],B[:,j])

In [15]:
R

,Терминатор,Робокоп,Рэмбо,Джеймс Бонд,Властелин Колец,Хоббит,Гарри Поттер,Американский Пирог,Мальчишник в Вегасе,Мстители,Супермен
Вася,9,0,0,6,4,3,5,7,8,6,5
Петя,0,0,0,8,10,0,8,5,6,2,0
Саша,8,0,5,7,4,0,5,7,8,8,5
Женя,5,2,0,4,10,0,9,5,0,0,0
Маша,0,0,2,5,7,0,0,9,0,4,0
Оля,2,3,0,0,0,7,9,0,5,0,3
Лена,5,0,3,8,8,0,0,7,0,2,4
Ваня,8,7,0,6,5,4,0,8,0,10,0
Ира,0,0,2,5,10,0,0,0,9,0,3


In [16]:
R_cap = pd.DataFrame(r_cap,u,f)

In [17]:
R_cap

,Терминатор,Робокоп,Рэмбо,Джеймс Бонд,Властелин Колец,Хоббит,Гарри Поттер,Американский Пирог,Мальчишник в Вегасе,Мстители,Супермен
Вася,8.048848,7.309958,4.796889,5.760791,4.177962,7.684160,7.860122,7.839558,8.552127,7.514806,4.954161
Петя,3.046008,1.987646,0.847493,5.208618,9.764011,4.410438,8.148371,5.532376,6.121796,1.301871,2.793734
Саша,8.621805,7.857801,5.172513,6.063992,4.186580,8.178133,8.237052,8.307074,9.059082,8.104169,5.274394
Женя,3.932968,2.869804,1.471324,5.545460,9.419278,5.109382,8.505475,6.143841,6.780322,2.281705,3.249257
Маша,4.850669,3.977825,2.359490,5.134498,7.010513,5.455772,7.577489,6.133112,6.738099,3.682205,3.490135
Оля,3.193650,2.378759,1.254923,4.314722,7.139853,4.055504,6.584926,4.829403,5.326357,1.948671,2.581329
Лена,4.518523,3.465927,1.900241,5.714409,9.047329,5.544307,8.649953,6.502258,7.164167,2.955778,3.533775
Ваня,7.502383,6.764716,4.410383,5.560010,4.408594,7.256883,7.651638,7.468547,8.152833,6.907683,4.675556
Ира,5.328660,4.206397,2.388906,6.275954,9.418424,6.308661,9.409843,7.275832,8.007532,3.721477,4.026874


In [18]:
print ('Start MSE:',M_1,'Finish MSE:',M_end[-1])

Start MSE: 4065.7950666666675 Finish MSE: 2.0605250655324685


In [19]:
print ('Start RMSE',np.sqrt(M_1),'Finish RMSE',np.sqrt(M_end[-1]))

Start RMSE 63.76358731020917 Finish RMSE 1.4354529130321443


In [20]:
n = (R.values != 0).sum()

In [21]:
L=0
for i in range(R.shape[0]):
    for j in range(R.shape[1]):
        if R.values[i,j]>0:
            L+=((R.values[i,j]-R_cap.values[i,j]) ** 2)/n

В итоге получили матрицу оценок с помощью SGD. Как видно, MSE сильно упал. Можно поиграться с параметрами, чтобы настроить качество. Как итог, выбираем фильмы с наибольшей оценкой и рекомендуем их пользователям! :) 

В заключение хочется сказать, что есть хорошая библиотека Surprise, где можно делать рекомендации проще, быстрее и различными способами.
Оставлю ссылку для ознакомления http://surpriselib.com/